# 9. Hyperparameter Tuning

> PyTorch 기반으로 여러 config들을 통해 학습할 때에 사용되는 parameter들을 실험자가 손 쉽게 지역 최적해를 구할 수 있도록 도와주는 Ray Tune 프레임워크로 최적화하는 방법을 학습합니다.  
이를 통해 Grid & Random, 그리고 Bayesian 같은 기본적인 Parameter Search 방법론들과 Ray Tune 모듈을 사용하여 PyTorch 딥러닝 프로젝트 코드 구성을 하는 방법을 익히게 됩니다.

<br>

## Reference

- [Pytorch와 Ray 같이 사용하기](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)

<br>

## 9.1 Hyperparameter Tuning

- 모델 스스로 학습하지 않는 값은 사람이 지정
  - learning rate, 모델의 크기, optimizer 등
- 하이퍼 파라메터에 의해서 값의 크게 좌우 될 때도 있음 (요즘은 그닥?)
- 마지막 0.01을 쥐어짜야 할 때 도전해볼만!

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1HGaKiTmoL8lUCVYOJaNOwWZOBBpzFqOO' width=400/>

- 출처 : https://content.iospress.com/articles/journal-of-intelligent-and-fuzzy-systems/ifs190033

<br>

- 가장 기본적인 방법
  - grid search : 값들을 일정한 범위 내에서 선택
  - random search : 값을 랜덤하게 선택
- 최근에는 베이지안 기반 기법들이 주도
  - 관련 논문 : BOHB (2018)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1xD62XXQohfURXwLal2hoW1xQDbJLgjsY' width=600/>

- 출처: https://dl.acm.org/doi/pdf/10.5555/2188385.2188395

<br>

## 9.2 Ray

- multi-node multi processing 지원 모듈
- ML/DL 의 병렬 처리를 위해 개발된 모듈
- 기본적으로 현재의 분산병렬 ML/DL 모듈의 표준
- Hyperparameter Search를 위한 다양한 모듈 제공

<br>

### 9.2.1 Example Code

```python
data_dir = os.path.abspath("./data")
load_data(data_dir)

# config 에 search space 지정
config = {
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

# 학습 스케줄링 알고리즘 지정
#  - 가망이 없는 하이퍼파라미터는 이후 스텝에서 변경 대상에서 제외됨
scheduler = ASHAScheduler(
    metric="loss", mode="min", max_t=max_num_epochs, grace_period=1,
    reduction_factor=2)

# 결과 출력 양식 지정
reporter = CLIReporter(
    metric_columns=["loss", "accuracy", "training_iteration"])

# 병렬 처리 양식으로 학습 시행
result = tune.run(
    partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
    config=config, num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter)
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1weM8YEKnkjpoUZktlPwjGw3wxpKZSWmc' width=600/>

<br>

## 9.3 실습: Ray-tune for Hyperparameter Turning

In [1]:
!pip install ray

     |████████████████████████████████| 51.0 MB 75 kB/s 
     |████████████████████████████████| 65 kB 3.0 MB/s 
     |████████████████████████████████| 1.3 MB 59.1 MB/s 
     |████████████████████████████████| 3.1 MB 73.0 MB/s 
     |████████████████████████████████| 72 kB 558 kB/s 
     |████████████████████████████████| 127 kB 74.7 MB/s 
     |████████████████████████████████| 78 kB 9.5 MB/s 
     |████████████████████████████████| 10.1 MB 58.1 MB/s 
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 142 kB 74.8 MB/s 
     |████████████████████████████████| 294 kB 72.7 MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=3083607aa7a58cc92acd1b962aced22e5c789298701eccd639f9cf933c2b0a88
  Stored in directory: /root/.cache/pip/wheels/e6/67/af/f1ad15974b8fd95f59a63dbf854483ebe5c7a46a93930798b8
Successfully built gpustat


In [2]:
!pip install tensorboardX # ray 가 내부적으로 tensorboardX 라는 모듈을 사용한다.

     |████████████████████████████████| 124 kB 14.9 MB/s 


In [3]:
!pip install wandb

     |████████████████████████████████| 1.6 MB 12.8 MB/s 
     |████████████████████████████████| 170 kB 65.2 MB/s 
     |████████████████████████████████| 133 kB 73.7 MB/s 
     |████████████████████████████████| 97 kB 9.7 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=012a26a9907c9976720d03ae68190e3fcb6f948767077e5f2f309dd02a26777b
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=a850ca16e2c03192321b0cf8cd70148ccfae697500967aaf39c0db1ae6b2f41a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [4]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

import wandb

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [5]:
def load_data(data_dir='./data'):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform
    )
    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform
    )

    return trainset, testset

In [6]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    # 모델이 학습하는 과정이 하나의 함수 안에 들어가 있어야 한다. (그래야 나중에 ray 에서 이것을 불러올 수 있다.)
    net = Net(config['l1'], config['l2'])

    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda:0'
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config['lr'], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, 'checkpoint')
        )
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config['batch_size']),
        shuffle=True,
        num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config['batch_size']),
        shuffle=True,
        num_workers=8
    )
    wandb.init(project='torch-turn', entity='zgotter')
    wandb.watch(net) # wandb tracking

    for epoch in range(10): # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0
            
        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()                

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        wandb.log({'val_loss': val_loss})
        wandb.log({'loss': loss})

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, 'checkpoint')
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=(correct / total))
    print('Finished Training')

In [8]:
def test_accuracy(net, device='cpu'):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total            

In [11]:
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.hyperopt import HyperOptSearch

def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):

    data_dir = os.path.abspath('./data')
    load_data(data_dir)
    
    config = {
        'l1': tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        'l2': tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        'lr': tune.loguniform(1e-4, 1e-1), # 0.1 단위 하이퍼파라미터는 로그균등분포에서 추출 (0.0001 ~ 0.1)
        'batch_size': tune.choice([2, 4, 6, 8, 16])
    }

    scheduler = ASHAScheduler(
        metric='loss',
        mode='min',
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2
    )

    reporter = CLIReporter(
        # parameter_columns=['l1', 'l2', 'lr', 'batch_size']
        metric_columns=['loss', 'accuracy', 'training_iteration']
    )

    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={'cpu': 2, 'gpu': gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))    

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)
    
    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(
        os.path.join(best_checkpoint_dir, "checkpoint")
    )
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    wandb.login(key="a0816ef4efd1811ed041758f0456fc4d72d47a8b")
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/data/cifar-10-python.tar.gz to /content/data
Files already downloaded and verified


2021-08-20 03:34:19,993	INFO services.py:1247 -- View the Ray dashboard at http://127.0.0.1:8265
2021-08-20 03:34:22,170	WARNING experiment.py:296 -- No name detected on trainable. Using DEFAULT.
2021-08-20 03:34:22,174	INFO registry.py:67 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 1.5/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/DEFAULT_2021-08-20_03-34-22
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+--------------+------+------+-------------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-------+--------------+------+------+-------------|
| DEFAULT_8260c_00000 | RUNNING  |       |            2 |   64 |    4 | 0.000217048 |
| DEFAULT_8260c_00001 | PENDING  |       |            4 |    8 |  256 | 0.00052147  |
| DEFAULT_8260c_00002 | PENDING  |       |            2 |  256 |  128 | 0.00181035  |
| DEFAULT_8260c_00003 | PENDING  |       |            8 |   64 |  256 | 0.000104527 |
| D

...

(pid=652) wandb: Tracking run with wandb version 0.12.0
(pid=652) wandb: Syncing run electric-fog-1
(pid=652) wandb:  View project at https://wandb.ai/zgotter/torch-turn
(pid=652) wandb:  View run at https://wandb.ai/zgotter/torch-turn/runs/2j53xzqe
(pid=652) wandb: Run data is saved locally in /root/ray_results/DEFAULT_2021-08-20_03-34-22/DEFAULT_8260c_00000_0_batch_size=2,l1=64,l2=4,lr=0.00021705_2021-08-20_03-34-22/wandb/run-20210820_033425-2j53xzqe
(pid=652) wandb: Run `wandb offline` to turn off syncing.
(pid=652) /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
(pid=652)   cpuset_checked))
(pid=652)

== Status ==
Memory usage on this node: 2.1/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/2.0 CPU_group_79efd6f806b05723e6dcfd20ff1c193f, 0.0/1.0 accelerator_type:P100, 0.0/2.0 CPU_group_0_79efd6f806b05723e6dcfd20ff1c193f)
Result logdir: /root/ray_results/DEFAULT_2021-08-20_03-34-22
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+--------------+------+------+-------------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-------+--------------+------+------+-------------|
| DEFAULT_8260c_00000 | RUNNING  |       |            2 |   64 |    4 | 0.000217048 |
| DEFAULT_8260c_00001 | PENDING  |       |            4 |    8 |  256 | 0.00052147  |
| DEFAULT_8260c_00002 | PENDING  |       |            2 |  256 |  128

2021-08-20 03:47:23,735	WARNING tune.py:508 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


(pid=652) [10, 20000] loss: 0.108


KeyboardInterrupt: ignored